# B. Classifieur Bayesien Naïf

In [4]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn import naive_bayes

# 1
Créez une fonction CBN(X,Y) qui prend en entrée des données X et des étiquettes Y et
qui renvoie une étiquette, pour chaque donnée, prédite à partir de la classe la plus
probable selon l’équation (1). Ici encore, on prend chaque donnée, une par une, comme
donnée de test et on considère toutes les données comme données d’apprentissage. Il est
conseillé de calculer d’abord les barycentres et les probabilités à priori P (ωk ) pour
chaque classe, puis de calculer les probabilités conditionnelles P (xi /ωk ) pour chaque
classe et chaque variable.

In [5]:
def CBN(x, y):
    """
    Naive bayes classifier
    
     x : data
     y : target (classe)
    """

    # Constantes
    nb_class = np.unique(y).size
    data_size = len(x)
    nb_features = x.shape[1]
    # probabilité de chaque classe
    proba_class = [e / float(len(x)) for e in Counter(y).values()] # trouver le ratio = probabilité

    # cbn_target represente les nouvelles etiquettes
    nbc_target = np.zeros(data_size, dtype=np.int)

    # 1 folds Cross validation
    # Choisi une observation parmis les donnees
    # Genere la matrice d'apprentissge en excluant l'observation precedente
    for i in range(data_size):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)

        # Regroupe les donnees par classe et calcule leurs barycentres (means)
        data_pclass = {} # dict -> classId: features of it
        mean_pclass = {} # moyenne de chaque classe
        for j in range(nb_class):
            data_pclass[j] = [e for k, e in enumerate(train) if train_target[k] == j]
            mean_pclass[j] = np.mean(a=data_pclass[j], axis=0)

        # Calcule des distances
        # dist represente la distance entre la donnee x et le barycentre pour chaque classe
        # dist_total est la somme des distances par classe
        dist = [abs(valid-barycentre) for barycentre in mean_pclass.values()]
        dist_total = np.sum(dist, axis=0)

        # Calcule de la probabilite PROP(P(xi/wk)P(wk))
        # xi/wk = une donne x avec la valeur xi pour la variable i de la classe wk
        '''P (xi /ωk ) : est la probabilité qu’une donnée x ait la valeur xi pour la variable i, si on connaît
        sa classe ωk. Nous allons calculer cette probabilité en calculant la distance entre la donnée xi et
        chaque barycentre des classes, divisée par la somme des distances entre cette donnée et chaque
        barycentre.'''
        tmp = (1-(dist/dist_total))
        # pour chaque classe, multiplier  P(xi /ωk ) par P(wk)
        for ii in range(len(tmp)): #classe
            for j in range(len(tmp[ii])): # xi
                tmp[ii][j] = tmp[ii][j]*proba_class[ii] # proba_class[ii] is 1/3 cuz we have 3 classes
        # produit
        pro = []
        for ii in range(len(tmp)):
            pro.append(tmp[ii].prod())
        pro
        
        
        nbc_target[i] = np.argmax(pro)

    return {'target': nbc_target}

# Teste!

In [6]:
# Jeu de donnee
from sklearn.datasets import load_iris
iris = load_iris ()
X = iris.data
Y = iris.target
nbc_target = CBN(x=X, y=Y)
nbc_target

{'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
        2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
        2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2])}

# 2.
La fonction CBN calcule une étiquette prédite pour chaque donnée. Modifiez la fonction
pour calculer et renvoyer l’erreur de prédiction : c’est à dire le pourcentage d’étiquettes
mal prédites. Testez sur les données Iris.

In [7]:
def validateur_malprediction(lis_1, lis_2):
    """ validateur_malprediction : Retourne le pourcentage d’étiquettes mal prédites
    """
    size = len(lis_1)
    if size != len(lis_2):
        logging.error("Liste de taille differente")
        return 0
    # 1) trouver le total d’étiquettes mal prédites
    _sum = 0
    for i in range(size):
        if lis_1[i] != lis_2[i]:
            _sum += 1
    # 2) retourner le facteur en %
    return _sum*100 / float(size)

In [8]:
def CBN_err(x, y):
    """
    Naive bayes classifier
    
     x : data
     y : target (classe)
    """

    # Constantes
    nb_class = np.unique(y).size
    data_size = len(x)
    nb_features = x.shape[1]
    # probabilité de chaque classe
    proba_class = [e / float(len(x)) for e in Counter(y).values()] # trouver le ratio = probabilité

    # cbn_target represente les nouvelles etiquettes
    nbc_target = np.zeros(data_size, dtype=np.int)

    # 1 folds Cross validation
    # Choisi une observation parmis les donnees
    # Genere la matrice d'apprentissge en excluant l'observation precedente
    for i in range(data_size):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)

        # Regroupe les donnees par classe et calcule leurs barycentres (means)
        data_pclass = {} # dict -> classId: features of it
        mean_pclass = {} # moyenne de chaque classe
        for j in range(nb_class):
            data_pclass[j] = [e for k, e in enumerate(train) if train_target[k] == j]
            mean_pclass[j] = np.mean(a=data_pclass[j], axis=0)

        # Calcule des distances
        # dist represente la distance entre la donnee x et le barycentre pour chaque classe
        # dist_total est la somme des distances par classe
        dist = [abs(valid-barycentre) for barycentre in mean_pclass.values()]

        dist_total = np.sum(dist, axis=0)
        # Calcule de la probabilite PROP(P(xi/wk)P(wk))
        # xi/wk = une donne x avec la valeur xi pour la variable i de la classe wk
        '''P (xi /ωk ) : est la probabilité qu’une donnée x ait la valeur xi pour la variable i, si on connaît
        sa classe ωk. Nous allons calculer cette probabilité en calculant la distance entre la donnée xi et
        chaque barycentre des classes, divisée par la somme des distances entre cette donnée et chaque
        barycentre.'''
        tmp = (1-(dist/dist_total))
        # pour chaque classe, multiplier  P(xi /ωk ) par P(wk)
        for ii in range(len(tmp)): #classe
            for j in range(len(tmp[ii])): # xi
                tmp[ii][j] = tmp[ii][j]*proba_class[ii] # proba_class[ii] is 1/3 cuz we have 3 classes

        # produit
        pro = []
        for ii in range(len(tmp)):
            pro.append(tmp[ii].prod())
        
        nbc_target[i] = np.argmax(pro)

    return {'target': nbc_target, 'error': validateur_malprediction(nbc_target, y)}

TESTE!

In [9]:
nbc = CBN_err(x=X, y=Y)
print("Target", nbc["target"])
print("Err: %.2f%%"% nbc["error"])


Target [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 2 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 1 2 2 2 2 2
 2 2]
Err: 12.67%


# 3.
Testez la fonction du Classifieur Bayesien Naïf inclut dans sklearn. Cette fonction
utilise une distribution Gaussienne au lieu des distances aux barycentres. Les résultats
sont-ils différents ?

In [10]:
def predic_cross_valid(algo, x, y):
    """train : cross validation pour l'algorithme de sklearn
        x = data
        y = target
    """
    # taille de jeu de donnees
    taille = len(x)

    predicted_target = np.zeros(taille, dtype=np.int)

    for i in range(taille):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)
        
        # trouver la classe
        algo.fit(train, train_target)
        predicted_target[i] = algo.predict(valid.reshape(1, -1))

    return {'target': predicted_target, 'error': validateur_malprediction(predicted_target, y)}


In [11]:
clf = naive_bayes.GaussianNB()
sklearn_nbc_target = predic_cross_valid(clf, X, Y)

print("CBN : %s", nbc['target'])
print("CBN pourcentage d’étiquettes mal prédites : %.1f%%"% nbc['error'])
def sep(n):
    print("_"*n)
sep(100)
print("Naive Bayes GaussianNB : %s"% sklearn_nbc_target['target'])
print("GaussianNB pourcentage d’étiquettes mal prédites : %.1f%%"% sklearn_nbc_target['error'])


CBN : %s [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 2 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 1 2 2 2 2 2
 2 2]
CBN pourcentage d’étiquettes mal prédites : 12.7%
____________________________________________________________________________________________________
Naive Bayes GaussianNB : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
GaussianNB pourcentage d’étiquettes mal prédites : 4.7%


pourcentages différents!

On va les rendrent égaux par suite!

# 4. 

Modifiez la fonction CBN pour qu’elle utilise une distribution Gaussienne pour les lois
de probabilités au lieu de simple distance au barycentre.

In [12]:
from math import exp, sqrt, pi
# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [13]:
def CBN_GAUSS(x, y):
    """
    Naive bayes classifier
    
     x : data
     y : target (classe)
    """

    # Constantes
    nb_class = np.unique(y).size
    data_size = len(x)
    nb_features = x.shape[1]
    # probabilité de chaque classe
    proba_class = [e / float(len(x)) for e in Counter(y).values()] # trouver le ratio = probabilité

    # cbn_target represente les nouvelles etiquettes
    nbc_target = np.zeros(data_size, dtype=np.int)

    # 1 folds Cross validation
    # Choisi une observation parmis les donnees
    # Genere la matrice d'apprentissge en excluant l'observation precedente
    for i in range(data_size):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)

        # Regroupe les donnees par classe et calcule leurs moyenne et ecart-type
        data_pclass = {} # dict -> classId: features of it
        mean_pclass = {} # moyenne de chaque classe
        std_pclass = {} # moyenne de chaque classe
        for j in range(nb_class):
            data_pclass[j] = [e for k, e in enumerate(train) if train_target[k] == j]
            mean_pclass[j] = np.mean(a=data_pclass[j], axis=0)
            std_pclass[j] = np.std(a=data_pclass[j], axis=0)
            
        
        # Calcule de la probabilite PROP(P(xi/wk), valid = xi
        prob = []
        for ii in range(nb_class): #classe
            tmp = []
            for indx in range(len(valid)):
                tmp.append(calculate_probability(valid[indx], mean_pclass[ii][indx], std_pclass[ii][indx]))
            prob.append(tmp)
       
        # pour chaque classe, multiplier  P(xi /ωk ) par P(wk)
        for ii in range(nb_class): #classe
            for j in range(len(prob[ii])): # xi
                prob[ii][j] = prob[ii][j]*proba_class[ii] # proba_class[ii] is 1/3 cuz we have 3 classes
        # produit
    
        product = []
        for ii in range(len(prob)):
            product.append(np.prod(prob[ii]))
        
        nbc_target[i] = np.argmax(product)

    return {'target': nbc_target, 'error': validateur_malprediction(nbc_target, y)}

In [14]:
nbc_barycentre = nbc
nbc_gauss = CBN_GAUSS(X, Y)
clf = naive_bayes.GaussianNB()
sklearn_nbc_target = predic_cross_valid(clf, X, Y)

print("++++ [ distance au barycentre]")
print("CBN : %s"% nbc_barycentre['target'])
print("CBN pourcentage d’étiquettes mal prédites : %.1f%%"% nbc_barycentre['error'])
sep(100)
print("++++ [distribution Gaussienne]")
print("CBN : %s"% nbc_gauss['target'])
print("CBN pourcentage d’étiquettes mal prédites : %.1f%%"% nbc_gauss['error'])
sep(100)
print("++++ [distribution Gaussienne GaussianNB de scikit-learn]")
print("Naive Bayes : %s"% sklearn_nbc_target['target'])
print("GaussianNB pourcentage d’étiquettes mal prédites : %.1f%%"% sklearn_nbc_target['error'])


++++ [ distance au barycentre]
CBN : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 2 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 1 2 2 2 2 2
 2 2]
CBN pourcentage d’étiquettes mal prédites : 12.7%
____________________________________________________________________________________________________
++++ [distribution Gaussienne]
CBN : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
CBN pourcentage d’étiquettes mal prédites : 4.7%
____________________________________________________________________________________________________
++++ [distrib